# Numerical instability

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider


def oscillating(x):
    """Vector-valued function that oscillates rapidly as x approaches 0."""
    result = np.zeros_like(x)  # Initialize an array of zeros with the same shape as x
    non_zero_mask = x != 0  # Mask for elements where x is not equal to 0
    result[non_zero_mask] = np.sin(1 / x[non_zero_mask])  # Apply sin(1/x) where x != 0
    return result


def d_oscillating(x):
    """Vector-valued exact derivative of the oscillating function."""
    result = np.zeros_like(x)  # Initialize an array of zeros with the same shape as x
    non_zero_mask = x != 0  # Mask for elements where x is not equal to 0
    result[non_zero_mask] = -np.cos(1 / x[non_zero_mask]) / (x[non_zero_mask] ** 2)  # Derivative where x != 0
    return result
    

def plot_oscillating(min=-0.1, max=0.1, steps=500):
    # Generate values of x around 0, excluding 0 to avoid division by zero
    x_values = np.linspace(min, max, steps)

    y_values = oscillating(x_values)
    dy_values = d_oscillating(x_values)

    # Create plots
    plt.figure(figsize=(14, 6))

    # Plot oscillating function
    plt.subplot(1, 2, 1)
    plt.plot(x_values, y_values, label='Oscillating Function', color='blue')
    plt.title('Oscillating Function $f(x) = sin(1/x)$')
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.grid(True)
    plt.legend()

    # Plot exact derivative
    plt.subplot(1, 2, 2)
    plt.plot(x_values, dy_values, label='Exact Derivative', color='orange')
    plt.title("Exact Derivative $f'(x) = -\\frac{cos(1/x)}{x^2}$")
    plt.xlabel('x')
    plt.ylabel("f'(x)")
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.show()

interact(
    plot_oscillating,
    min=FloatSlider(min=-0.1, max=-0.01, step=0.01, value=-0.1, description='Min'),
    max=FloatSlider(min=0.01, max=0.1, step=0.01, value=0.1, description='Max'),
    steps=IntSlider(min=100, max=1000, step=100, value=500, description='Steps')
)

interactive(children=(FloatSlider(value=-0.1, description='Min', max=-0.01, min=-0.1, step=0.01), FloatSlider(…

<function __main__.plot_oscillating(min=-0.1, max=0.1, steps=500)>

# Numerical Instability with the Central Difference

In [2]:
import numpy as np


def cdiff(func, x, h=1e-3):
    """Centered difference approximation of the derivative."""
    return (func(x + h) - func(x - h)) / (2 * h)


def oscillating(x):
    """Function that oscillates rapidly as x approaches 0."""
    return np.sin(1/x) if x != 0 else 0


def d_oscillating(x):
    """Exact derivative of the oscillating function."""
    if x != 0:
        return -np.cos(1/x) / (x**2)
    else:
        return 0  # Not defined, but can be treated as 0 for comparison


# Test values close to 0
x_values = [0.1, 0.01, 0.001, 0.0001, 0.00001]

# Compute and compare the derivatives
print(f"{'x':>12} | {'Numerical Derivative':>20} | {'Exact Derivative':>20} | {'Abs Error':>20}")
print("-" * 80)
for x in x_values:
    numerical_derivative = cdiff(oscillating, x)
    exact_deriv = d_oscillating(x)
    
    # Calculate the relative error
    if exact_deriv != 0:  # Avoid division by zero for relative error
        abs_error = np.abs((numerical_derivative - exact_deriv))
    else:
        abs_error = np.nan  # Not defined for comparison

    print(f"{x:>12} | {numerical_derivative:>20.6f} | {exact_deriv:>20.6f} | {abs_error:>20.6f}")


           x | Numerical Derivative |     Exact Derivative |            Abs Error
--------------------------------------------------------------------------------
         0.1 |            83.721363 |            83.907153 |             0.185790
        0.01 |           555.383031 |         -8623.188723 |          9178.571754
       0.001 |          -233.885903 |       -562379.076291 |        562145.190388
      0.0001 |          -884.627816 |      95215536.825901 |      95216421.453717
       1e-05 |          -736.979383 |    9993608074.376921 |    9993608811.356304
